In [1]:
import serial
from serial.tools import list_ports
import yaml
import cv2
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import os
matplotlib.use('TkAgg')

ModuleNotFoundError: No module named 'yaml'

In [4]:
# list the available serial ports

ports = serial.tools.list_ports.comports()

print([port.name for port in ports]) 

['ttyACM0', 'ttyAMA0']


In [5]:
# open the serial port
ser = serial.Serial('/dev/ttyACM0', 115200)  # baudrate = 115200

In [6]:
ser.is_open # did it really open? 

True

In [7]:
ser.write(b'G90\n') # explicitly set absolute positioning

4

In [8]:
ser.write(b'G1 X100 Y100 F6000\n') # go to a position!

19

In [67]:
# now grab an image from the usb microscop & show it
videoCaptureObject = cv2.VideoCapture(1)
result = True
while(result):
    ret,frame = videoCaptureObject.read()
    # cv2.imwrite("test.jpg",frame)
    result = False
videoCaptureObject.release()
cv2.destroyAllWindows()
w,h,z = frame.shape
print(h)
plt.imshow(frame)
plt.title('frame capture')
plt.show()
cx, cy = plt.ginput(1)[0] # [number of clicks][single item list]
print(cx)

640
459.8973509933775


In [8]:
# load in the calibration file
with open("./calibration/distance-calibration.yml", "r") as stream:
    try:
        dcal = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
# grab the x & y values
rx = dcal['z_50']['x']
ry = dcal['z_50']['y']
print(ry)

7.54


In [69]:
# translate from pixels to real world
# the camera is rotated relative to the bed, so +x in cam space is -y on bed
# check this...
dy = np.interp(cx,[0,w],[-rx/2, rx/2])
dx = np.interp(cy, [0,h], [ry/2,-ry/2])

In [70]:
# now format to send over serial
dx = "{0:.2f}".format(dx)
dy = "{0:.2f}".format(dy)
print(dx)
print(dy)

-1.49
4.54


In [ ]:
## Send gcode
command = f'G1 X{dx} Y{dy} F6000\n'
ser.write(b'G91\n') # set relative positioning

4

In [72]:
ser.write(bytes(command, encoding='utf-8')) # send command

22

In [74]:
ser.write(b'G90\n') # restore absolute positioning

4

In [32]:
# image (a section of) the bed, stepping by the microscope viewport size
# save out each image
import os
for x in np.arange(125,135,rx):
    if not os.path.exists(f'./img_bed/{x}'):
        os.makedirs(f'./img_bed/{x}')
    for y in np.arange(150, 170, ry):
        cmd = f'G0 X{x} Y{y} F6000\n'
        ser.write(bytes(cmd, encoding='utf-8'))
        videoCaptureObject = cv2.VideoCapture(1)
        result = True
        while(result):
            ret,frame = videoCaptureObject.read()
            cv2.imwrite(f'./img_bed/{x}/{y}.jpg',frame)
            result = False
        videoCaptureObject.release()
        cv2.destroyAllWindows()

In [35]:
# now load in on horizontal row of images to hconcat them
images = []
fnames = []
folder = '/home/pi/duckweed/img_bed/134.92'
for filename in sorted(os.listdir(folder)):
    fnames.append(os.path.join(folder,filename))
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
        images.append(img)

In [36]:
concat = cv2.vconcat(images)
cv2.imwrite('concat.jpg', concat)

True